# Inroduction


In [1]:
!pip install pathway bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 66.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 50.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import pathway as pw
import bokeh.plotting
import panel as pn

# Step 1: Importing and Preprocessing the Data

In [222]:
df = pd.read_csv('/content/sample_data/dataset.csv')
# Define the mapping dictionaries
vehicle_type_mapping = {
    'cycle' : 0.2,
    'car': 1.0,
    'bike': 0.5,
    'truck': 1.5
}

traffic_mapping = {
    'low': 0.3,
    'average': 0.6,
    'high': 1.0
}

# Apply the mapping to the DataFrame columns
# Use .loc to avoid SettingWithCopyWarning
df.loc[:, 'VehicleType'] = df['VehicleType'].replace(vehicle_type_mapping)
df.loc[:, 'TrafficConditionNearby'] = df['TrafficConditionNearby'].replace(traffic_mapping)
cols_to_float = ['Occupancy', 'Capacity', 'QueueLength', 'VehicleType', 'TrafficConditionNearby', 'IsSpecialDay']
df[cols_to_float] = df[cols_to_float].astype(float)



# Display the updated DataFrame with the new columns
df.head()

/tmp/ipython-input-222-3322261796.py:18: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'VehicleType'] = df['VehicleType'].replace(vehicle_type_mapping)
/tmp/ipython-input-222-3322261796.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.loc[:, 'TrafficConditionNearby'] = df['TrafficConditionNearby'].replace(traffic_mapping)


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime
0,0,BHMBCCMKT01,577.0,26.144536,91.736172,61.0,1.0,0.3,1.0,0.0,04-10-2016,07:59:00
1,1,BHMBCCMKT01,577.0,26.144536,91.736172,64.0,1.0,0.3,1.0,0.0,04-10-2016,08:25:00
2,2,BHMBCCMKT01,577.0,26.144536,91.736172,80.0,1.0,0.3,2.0,0.0,04-10-2016,08:59:00
3,3,BHMBCCMKT01,577.0,26.144536,91.736172,107.0,1.0,0.3,2.0,0.0,04-10-2016,09:32:00
4,4,BHMBCCMKT01,577.0,26.144536,91.736172,150.0,0.5,0.3,2.0,0.0,04-10-2016,09:59:00


In [172]:
# Combine the 'LastUpdatedDate' and 'LastUpdatedTime' columns into a single datetime column
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'],
                                  format='%d-%m-%Y %H:%M:%S')

unique_ids = df["SystemCodeNumber"].unique()

# Create a dictionary to hold the 14 dataframes
lot_dataframes = {lot_id: df[df["SystemCodeNumber"] == lot_id].copy() for lot_id in unique_ids}

# Sort the DataFrame by the new 'Timestamp' column and reset the index
for lot_id in lot_dataframes:
    lot_dataframes[lot_id] = lot_dataframes[lot_id].sort_values(by="Timestamp").reset_index(drop=True)

lot_dataframes["BHMBCCMKT01"].head()


,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577.0,26.144536,91.736172,61.0,1.0,0.33,1.0,0.0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577.0,26.144536,91.736172,64.0,1.0,0.33,1.0,0.0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577.0,26.144536,91.736172,80.0,1.0,0.33,2.0,0.0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577.0,26.144536,91.736172,107.0,1.0,0.33,2.0,0.0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577.0,26.144536,91.736172,150.0,0.5,0.33,2.0,0.0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [173]:
import ipywidgets as widgets
from IPython.display import display

# Create dropdown from keys
lot_selector = widgets.Dropdown(
    options=list(lot_dataframes.keys()),
    description='Select Lot:',
    value=list(lot_dataframes.keys())[0]
)

display(lot_selector)



Dropdown(description='Select Lot:', options=('BHMBCCMKT01', 'BHMBCCTHL01', 'BHMEURBRD01', 'BHMMBMMBX01', 'BHMN…

In [226]:
test_df = lot_dataframes[lot_selector.value]
lot_name = test_df["SystemCodeNumber"].iloc[0]

test_df.head()



,ID,SystemCodeNumber,Capacity,Latitude,Longitude,Occupancy,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay,LastUpdatedDate,LastUpdatedTime,Timestamp
0,0,BHMBCCMKT01,577.0,26.144536,91.736172,61.0,1.0,0.33,1.0,0.0,04-10-2016,07:59:00,2016-10-04 07:59:00
1,1,BHMBCCMKT01,577.0,26.144536,91.736172,64.0,1.0,0.33,1.0,0.0,04-10-2016,08:25:00,2016-10-04 08:25:00
2,2,BHMBCCMKT01,577.0,26.144536,91.736172,80.0,1.0,0.33,2.0,0.0,04-10-2016,08:59:00,2016-10-04 08:59:00
3,3,BHMBCCMKT01,577.0,26.144536,91.736172,107.0,1.0,0.33,2.0,0.0,04-10-2016,09:32:00,2016-10-04 09:32:00
4,4,BHMBCCMKT01,577.0,26.144536,91.736172,150.0,0.5,0.33,2.0,0.0,04-10-2016,09:59:00,2016-10-04 09:59:00


In [208]:
# Define the schema for the streaming data using Pathway
# This schema specifies the expected structure of each data row in the stream

class ParkingSchema(pw.Schema):
    SystemCodeNumber: str  # Unique identifier for the parking lot
    Timestamp : str
    LastUpdatedTime: str
    LastUpdatedDate: str
    Occupancy: float   # Number of occupied parking spots
    Capacity: float    # Total parking capacity at the location
    VehicleType: float # Type of vehicle (e.g., car, bike, truck)
    TrafficConditionNearby: float
    QueueLength : float
    IsSpecialDay : float



In [228]:
# Save a simplified version of the selected parking lot's data for streaming
# This will be used for real-time simulations with the pricing model
test_df[['SystemCodeNumber', 'Timestamp', 'LastUpdatedTime','LastUpdatedDate','Occupancy', 'Capacity', 'VehicleType', 'TrafficConditionNearby' , 'QueueLength', 'IsSpecialDay']].to_csv('parking_stream.csv', index=False)

# NOTE: Only basic features are used here. Model 2 and 3 should include more variables
# like traffic level, queue length, and competitor prices.
check = pd.read_csv('parking_stream.csv')

check.head()

,SystemCodeNumber,Timestamp,LastUpdatedTime,LastUpdatedDate,Occupancy,Capacity,VehicleType,TrafficConditionNearby,QueueLength,IsSpecialDay
0,BHMBCCMKT01,2016-10-04 07:59:00,07:59:00,04-10-2016,61.0,577.0,1.0,0.33,1.0,0.0
1,BHMBCCMKT01,2016-10-04 08:25:00,08:25:00,04-10-2016,64.0,577.0,1.0,0.33,1.0,0.0
2,BHMBCCMKT01,2016-10-04 08:59:00,08:59:00,04-10-2016,80.0,577.0,1.0,0.33,2.0,0.0
3,BHMBCCMKT01,2016-10-04 09:32:00,09:32:00,04-10-2016,107.0,577.0,1.0,0.33,2.0,0.0
4,BHMBCCMKT01,2016-10-04 09:59:00,09:59:00,04-10-2016,150.0,577.0,0.5,0.33,2.0,0.0


In [211]:
# Load the data as a simulated stream using Pathway's replay_csv function
# This replays the CSV data at a controlled input rate to mimic real-time streaming
# input_rate=1000 means approximately 1000 rows per second will be ingested into the stream.

data = pw.demo.replay_csv("parking_stream.csv", schema=ParkingSchema, input_rate=1000)

In [212]:
# Define the datetime format to parse the 'Timestamp' column
fmt = "%Y-%m-%d %H:%M:%S"

# Add new columns to the data stream:
# - 't' contains the parsed full datetime
# - 'day' extracts the date part and resets the time to midnight (useful for day-level aggregations)
data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

# Step 2: Making a simple pricing function

In [204]:
def normalize(value, min_val, max_val):
    return (value - min_val) / (max_val - min_val + 1e-5)

In [213]:
import datetime

# Constants
BASE_PRICE = 10
ALPHA = 1.0
BETA = 0.5
GAMMA = 0.3
DELTA = 0.8
EPSILON = 1.0
LAMBDA = 0.5

# Step 1: 30-minute tumbling window and aggregation
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.day,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=30)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        occ=pw.reducers.max(pw.this.Occupancy),
        cap=pw.reducers.max(pw.this.Capacity),
        qLength=pw.reducers.max(pw.this.QueueLength),
        specialDay=pw.reducers.max(pw.this.IsSpecialDay),
        vehicleType=pw.reducers.max(pw.this.VehicleType),
        traffic=pw.reducers.max(pw.this.TrafficConditionNearby),
    )
)

# Step 2: Calculate raw demand
delta_window = delta_window.with_columns(
    raw_demand=(
        ALPHA * (pw.this.occ / (pw.this.cap + 1e-5)) +
        BETA * pw.this.qLength -
        GAMMA * pw.this.traffic +
        DELTA * pw.this.specialDay +
        EPSILON * pw.this.vehicleType
    )
)

# Step 3: Normalize demand and calculate price in separate steps
delta_window = delta_window.with_columns(
    norm_demand=pw.this.raw_demand / (ALPHA + BETA * 10 + DELTA + EPSILON * 3 + 1e-5)
)

delta_window = delta_window.with_columns(
    price=pw.apply(lambda d: round(BASE_PRICE * (1 + LAMBDA * d), 2), pw.this.norm_demand)
)


# Step 3: Visualizing Daily Price Fluctuations with a Bokeh Plot

**Note:** The Bokeh plot in the next cell will only be generated after you run the `pw.run()` cell (i.e., the final cell).


In [214]:
# Activate the Panel extension to enable interactive visualizations
pn.extension()

# Define a custom Bokeh plotting function that takes a data source (from Pathway) and returns a figure
def price_plotter(source):
    # Create a Bokeh figure with datetime x-axis
    fig = bokeh.plotting.figure(
        height=400,
        width=800,
        title=lot_name + " : Parking Price ",
        x_axis_type="datetime",  # Ensure time-based data is properly formatted on the x-axis
    )
    # Plot a line graph showing how the price evolves over time
    fig.line("t", "price", source=source, line_width=2, color="navy")

    # Overlay red circles at each data point for better visibility
    fig.scatter("t", "price", source=source, size=6, color="red")

    return fig

# Use Pathway's built-in .plot() method to bind the data stream (delta_window) to the Bokeh plot
# - 'price_plotter' is the rendering function
# - 'sorting_col="t"' ensures the data is plotted in time order
viz = delta_window.plot(price_plotter, sorting_col="t")

# Create a Panel layout and make it servable as a web app
# This line enables the interactive plot to be displayed when the app is served
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [215]:
# Start the Pathway pipeline execution in the background
# - This triggers the real-time data stream processing defined above
# - %%capture --no-display suppresses output in the notebook interface

%%capture --no-display
pw.run()

Output()

ERROR:pathway_engine.connectors:Parse error: cannot create a field "Timestamp" with type str from value None. Original error: TypeError: cannot create an object of type String from value None
ERROR:pathway_engine.connectors:Parse error: cannot create a field "Timestamp" with type str from value None. Original error: TypeError: cannot create an object of type String from value None
ERROR:pathway_engine.connectors:Parse error: cannot create a field "Timestamp" with type str from value None. Original error: TypeError: cannot create an object of type String from value None
ERROR:pathway_engine.connectors:Parse error: cannot create a field "Timestamp" with type str from value None. Original error: TypeError: cannot create an object of type String from value None
ERROR:pathway_engine.connectors:Parse error: cannot create a field "Timestamp" with type str from value None. Original error: TypeError: cannot create an object of type String from value None
ERROR:pathway_engine.connectors:Parse er

KeyboardInterrupt: 